In [ ]:
# default_exp pds

## PDS related stuff for pyuvis
> Like index files etc.

These are provided as stored on PDS by `planetarypy` but we do helpful filtering in here on top of those catalogs.

In [ ]:
# export
from datetime import date as dtdate
from datetime import timedelta
from pathlib import Path

from yarl import URL

import hvplot.pandas  # noqa
from nbverbose.showdoc import show_doc
from planetarypy.pds.apps import find_indexes, get_index
from planetarypy import utils

In [ ]:
# export
class CatalogFilter:
    "useful to make class to keep the base catalog in memory"

    def __init__(self, date: str = None):
        self.date = date
        df = get_index("cassini.uvis", "index")
        df["product_id"] = df.FILE_NAME.map(lambda x: Path(x).stem)
        df["volume"] = df.FILE_NAME.map(lambda x: x.split("/")[1])
        new_cols = ["product_id", "volume"] + list(df.columns)[:-2]
        df = df[new_cols]
        df.set_index("product_id", inplace=True)
        df.OBSERVATION_TYPE.fillna("UNKNOWN", inplace=True)
        self.df = df

    @property
    def date(self):
        return self._date

    @date.setter
    def date(self, value: str):
        date = value.replace("_", "-")
        tokens = date.split("-")
        if len(tokens) == 2:
            date = utils.nasa_date_to_datetime(date)
        elif len(tokens) == 3:
            tokens = [int(i) for i in tokens]
            date = dtdate(*tokens)
        self._date = date

    def set_next_day(self):
        next_day = self.date + timedelta(days=1)
        self.date = next_day.isoformat()[:10]
        
    @property
    def isodate(self):
        return self.date.isoformat()[:10]
    
    @property
    def nasadate(self):
        return utils.iso_to_nasa_date(self.isodate)

    @property
    def piddate(self):
        return self.nasadate.replace("-","_")
    @property
    def fuvdate(self):
        return "FUV"+self.piddate
    
    @property
    def euvdate(self):
        return "EUV"+self.piddate

    @property
    def uv(self):
        "return E/F_UV only"
        return self.df[self.df.index.map(lambda x: x[:3] in ["EUV", "FUV"])]

    def _get_clean_date_filter(self, date: str = None):
        date = self.date if date is None else date
        if "-" in date:
            date = date.replace("-", "_")
        return date

    def get_fuv_date(self, date: str = None):  # datestring like yyyy-jjj, e.g. 2010-198
        date = self.date if date is None else date
        return self.df[self.df.index.str.startswith(self.fuvdate)]

    def get_euv_date(self, date: str = None):  # datestring like yyyy-jjj
        date = self.date if date is None else date
        return self.df[self.df.index.str.startswith(self.euvdate)]

    @property
    def ustare_stars(self):
        return self.uv.query("OBSERVATION_TYPE=='USTARE' and TARGET_NAME=='STAR'")

    @property
    def target_calib(self):
        return self.uv.query("TARGET_NAME=='CALIB'")

    def get_ustare_stars_date(
        self,
        date: str = None,  # datestring like yyyy-jjj
        uvtype: str = "",  # add EUV/FUV filter
    ):
        date = self.date if date is None else date
        _filter = uvtype + self.nasadate
        df = self.ustare_stars
        return df[df.index.str.contains(_filter)]

In [ ]:
date = "2002-198"

In [ ]:
cat = CatalogFilter(date)
cat.date

Stored index is up-to-date.


datetime.datetime(2002, 7, 17, 0, 0)

In [ ]:
cat.nasadate

'2002-198'

In [ ]:
cat.isodate

'2002-07-17'

In [ ]:
cat.df.groupby(["OBSERVATION_TYPE", "TARGET_NAME"]).size().index

MultiIndex([(  'CALIB',      'CALIB'),
            (  'CALIB', 'SOLAR WIND'),
            (  'CALIB',       'STAR'),
            (  'CALIB',        'SUN'),
            (  'CALIB',        'UNK'),
            ( 'UCSTAR',    'JUPITER'),
            ('UFPSCAN',    'JUPITER'),
            ('UFPSCAN', 'SOLAR WIND'),
            (  'UHDAC',      'DIONE'),
            (  'UHDAC',      'EARTH'),
            (  'UHDAC',  'ENCELADUS'),
            (  'UHDAC', 'EPIMETHEUS'),
            (  'UHDAC',   'HYPERION'),
            (  'UHDAC',    'IAPETUS'),
            (  'UHDAC',    'JUPITER'),
            (  'UHDAC',      'MIMAS'),
            (  'UHDAC',     'PHOEBE'),
            (  'UHDAC',       'RHEA'),
            (  'UHDAC',     'SATURN'),
            (  'UHDAC', 'SOLAR WIND'),
            (  'UHDAC',       'STAR'),
            (  'UHDAC',     'TETHYS'),
            (  'UHDAC',      'TITAN'),
            (  'UHDAC',        'UNK'),
            ('UHIGHSN',    'JUPITER'),
            (   'UMAP',  

In [ ]:
cat.uv.tail()

,volume,FILE_NAME,OBSERVATION_TYPE,START_TIME,STOP_TIME,TARGET_NAME,DATA_SET_ID,SPACECRAFT_CLOCK_START_COUNT,SPACECRAFT_CLOCK_STOP_COUNT,INTEGRATION_DURATION,...,SUB_SOLAR_LONGITUDE,SUB_SPACECRAFT_LATITUDE,SUB_SPACECRAFT_LONGITUDE,PHASE_ANGLE,EMISSION_ANGLE,SOLAR_INCIDENCE_ANGLE,CENTRAL_BODY_DISTANCE,DWELL_TIME,H_LEVEL,D_LEVEL
product_id,,,,,,,,,,,,,,,,,,,,,
FUV2017_258_07_15,COUVIS_0060,/COUVIS_0060/DATA/D2017_258/FUV2017_258_07_15.LBL,UNKNOWN,2017-09-15 07:15:04.442,2017-09-15 10:31:51.692,SOLAR WIND,CO-S-UVIS-2-SPEC-V1.4,1/1884155091.032,UNK,2.625,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999,-999.0,-999,<NA>,<NA>
EUV2017_257_22_39,COUVIS_0060,/COUVIS_0060/DATA/D2017_257/EUV2017_257_22_39.LBL,UNKNOWN,2017-09-14 22:39:54.016,2017-09-15 03:12:57.766,SOLAR WIND,CO-S-UVIS-2-CUBE-V1.4,1/1884124180.128,UNK,8191.875,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999,-999.0,-999,<NA>,<NA>
EUV2017_258_04_31,COUVIS_0060,/COUVIS_0060/DATA/D2017_258/EUV2017_258_04_31.LBL,UNKNOWN,2017-09-15 04:31:53.005,2017-09-15 09:04:56.755,SOLAR WIND,CO-S-UVIS-2-CUBE-V1.4,1/1884145299.160,UNK,8191.875,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999,-999.0,-999,<NA>,<NA>
EUV2017_258_07_00,COUVIS_0060,/COUVIS_0060/DATA/D2017_258/EUV2017_258_07_00.LBL,UNKNOWN,2017-09-15 07:00:27.697,2017-09-15 07:13:40.447,SOLAR WIND,CO-S-UVIS-2-SPEC-V1.4,1/1884154214.096,UNK,2.625,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999,-999.0,-999,<NA>,<NA>
EUV2017_258_07_15,COUVIS_0060,/COUVIS_0060/DATA/D2017_258/EUV2017_258_07_15.LBL,UNKNOWN,2017-09-15 07:15:04.442,2017-09-15 10:31:51.692,SOLAR WIND,CO-S-UVIS-2-SPEC-V1.4,1/1884155091.032,UNK,2.625,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999,-999.0,-999,<NA>,<NA>


In [ ]:
cat.ustare_stars.index

Index(['FUV2002_128_08_01', 'FUV2002_128_09_01', 'FUV2002_155_18_00',
       'FUV2002_155_18_09', 'FUV2002_155_18_34', 'FUV2002_155_18_58',
       'FUV2002_155_19_06', 'FUV2002_155_19_11', 'FUV2002_155_19_15',
       'FUV2002_155_19_19',
       ...
       'EUV2003_314_02_03', 'EUV2003_314_02_06', 'EUV2003_314_02_21',
       'EUV2003_314_02_24', 'EUV2003_314_02_28', 'EUV2003_314_02_54',
       'EUV2003_314_02_58', 'EUV2003_314_03_01', 'FUV2004_219_16_19',
       'EUV2004_219_16_19'],
      dtype='object', name='product_id', length=348)

In [ ]:
cat.date

'2002-198'

In [ ]:
cat.get_ustare_stars_date().index

Index(['FUV2002_198_03_26', 'FUV2002_198_04_31', 'FUV2002_198_05_28',
       'FUV2002_198_06_24', 'FUV2002_198_07_20', 'FUV2002_198_08_16',
       'FUV2002_198_09_12', 'FUV2002_198_10_09', 'FUV2002_198_11_05',
       'FUV2002_198_12_01', 'FUV2002_198_12_57', 'FUV2002_198_13_14',
       'FUV2002_198_13_53', 'EUV2002_198_03_26', 'EUV2002_198_04_31',
       'EUV2002_198_05_28', 'EUV2002_198_06_24', 'EUV2002_198_07_20',
       'EUV2002_198_08_16', 'EUV2002_198_09_12', 'EUV2002_198_10_09',
       'EUV2002_198_11_05', 'EUV2002_198_12_01', 'EUV2002_198_12_57',
       'EUV2002_198_13_14', 'EUV2002_198_13_53'],
      dtype='object', name='product_id')

In [ ]:
cat.uv.head()

,volume,FILE_NAME,OBSERVATION_TYPE,START_TIME,STOP_TIME,TARGET_NAME,DATA_SET_ID,SPACECRAFT_CLOCK_START_COUNT,SPACECRAFT_CLOCK_STOP_COUNT,INTEGRATION_DURATION,...,SUB_SOLAR_LONGITUDE,SUB_SPACECRAFT_LATITUDE,SUB_SPACECRAFT_LONGITUDE,PHASE_ANGLE,EMISSION_ANGLE,SOLAR_INCIDENCE_ANGLE,CENTRAL_BODY_DISTANCE,DWELL_TIME,H_LEVEL,D_LEVEL
product_id,,,,,,,,,,,,,,,,,,,,,
FUV1999_007_16_57,COUVIS_0001,/COUVIS_0001/DATA/D1999_007/FUV1999_007_16_57.LBL,USTARE,1999-01-07 16:57:21.952,1999-01-07 17:00:57.952,<NA>,CO-J-UVIS-2-SPEC-V1.2,1/1294419723.000,UNK,4.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999,-999.0,-999,<NA>,<NA>
FUV1999_007_17_01,COUVIS_0001,/COUVIS_0001/DATA/D1999_007/FUV1999_007_17_01.LBL,USTARE,1999-01-07 17:01:11.950,1999-01-07 17:04:47.950,<NA>,CO-J-UVIS-2-SPEC-V1.2,1/1294419953.000,UNK,4.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999,-999.0,-999,<NA>,<NA>
EUV1999_007_17_05,COUVIS_0001,/COUVIS_0001/DATA/D1999_007/EUV1999_007_17_05.LBL,USTARE,1999-01-07 17:05:01.949,1999-01-07 17:08:37.949,<NA>,CO-J-UVIS-2-SPEC-V1.2,1/1294420183.000,UNK,4.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999,-999.0,-999,<NA>,<NA>
EUV1999_007_17_08,COUVIS_0001,/COUVIS_0001/DATA/D1999_007/EUV1999_007_17_08.LBL,USTARE,1999-01-07 17:08:51.947,1999-01-07 17:12:27.947,<NA>,CO-J-UVIS-2-SPEC-V1.2,1/1294420413.000,UNK,4.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999,-999.0,-999,<NA>,<NA>
FUV1999_010_07_16,COUVIS_0001,/COUVIS_0001/DATA/D1999_010/FUV1999_010_07_16.LBL,USTARE,1999-01-10 07:16:50.522,1999-01-10 09:04:10.522,<NA>,CO-J-UVIS-2-SPEC-V1.2,1/1294644093.000,UNK,10.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999,-999.0,-999,<NA>,<NA>


In [ ]:
df.columns

Index(['FILE_NAME', 'OBSERVATION_TYPE', 'START_TIME', 'STOP_TIME',
       'TARGET_NAME', 'DATA_SET_ID', 'SPACECRAFT_CLOCK_START_COUNT',
       'SPACECRAFT_CLOCK_STOP_COUNT', 'INTEGRATION_DURATION',
       'COMPRESSION_TYPE', 'HI_VOLTAGE_POWER_SUPPLY_STATE',
       'OCCULTATION_PORT_STATE', 'SLIT_STATE', 'TEST_PULSE_STATE', 'ODC_ID',
       'RIGHT_ASCENSION', 'DECLINATION', 'SUB_SOLAR_LATITUDE',
       'SUB_SOLAR_LONGITUDE', 'SUB_SPACECRAFT_LATITUDE',
       'SUB_SPACECRAFT_LONGITUDE', 'PHASE_ANGLE', 'EMISSION_ANGLE',
       'SOLAR_INCIDENCE_ANGLE', 'CENTRAL_BODY_DISTANCE', 'DWELL_TIME',
       'H_LEVEL', 'D_LEVEL'],
      dtype='object')

In [ ]:
get_index("cassini.uvis", "ring_summary").head()

,VOLUME_ID,FILE_SPECIFICATION_NAME,RING_OBSERVATION_ID,MINIMUM_RIGHT_ASCENSION,MAXIMUM_RIGHT_ASCENSION,MINIMUM_DECLINATION,MAXIMUM_DECLINATION,MINIMUM_RING_RADIUS,MAXIMUM_RING_RADIUS,FINEST_RING_INTERCEPT_RESOLUTION,...,RING_CENTER_DISTANCE,SUB_SOLAR_RING_LONGITUDE,SUB_OBSERVER_RING_LONGITUDE,RING_CENTER_PHASE_ANGLE,RING_CENTER_INCIDENCE_ANGLE,RING_CENTER_NORTH_BASED_INCIDENCE_ANGLE,RING_CENTER_EMISSION_ANGLE,RING_CENTER_NORTH_BASED_EMISSION_ANGLE,SOLAR_RING_OPENING_ANGLE,OBSERVER_RING_OPENING_ANGLE
0,COUVIS_0006,DATA/D2004_002/EUV2004_002_12_16.LBL,S/CO/UVIS/2004-002T12-16/EUV,34.154493,37.815141,9.679762,9.712415,1074925.366,3066570.289,87709.0,...,8.672435e+07,283.109,215.938,62.786,64.404,115.596,73.661,106.339,-25.596,-16.34
1,COUVIS_0006,DATA/D2004_002/EUV2004_002_12_32.LBL,S/CO/UVIS/2004-002T12-32/EUV,34.155557,37.817313,9.780730,9.813480,521741.812,2900680.924,87173.0,...,8.671942e+07,283.109,215.938,62.787,64.404,115.596,73.661,106.339,-25.596,-16.34
2,COUVIS_0006,DATA/D2004_002/EUV2004_002_12_47.LBL,S/CO/UVIS/2004-002T12-47/EUV,34.154107,37.816935,9.877249,9.909878,64757.681,2839690.546,86667.0,...,8.671448e+07,283.110,215.938,62.787,64.404,115.596,73.661,106.339,-25.596,-16.34
3,COUVIS_0006,DATA/D2004_002/EUV2004_002_13_03.LBL,S/CO/UVIS/2004-002T13-03/EUV,34.155193,37.819152,9.978176,10.010898,541745.355,2872984.474,86145.0,...,8.670955e+07,283.110,215.938,62.787,64.404,115.596,73.661,106.339,-25.596,-16.34
4,COUVIS_0006,DATA/D2004_002/EUV2004_002_13_18.LBL,S/CO/UVIS/2004-002T13-18/EUV,34.157243,37.822332,10.077845,10.110536,1068459.125,2999016.495,85635.0,...,8.670461e+07,283.111,215.938,62.788,64.404,115.596,73.661,106.339,-25.596,-16.34


In [ ]:
df.head()

,FILE_NAME,OBSERVATION_TYPE,START_TIME,STOP_TIME,TARGET_NAME,DATA_SET_ID,SPACECRAFT_CLOCK_START_COUNT,SPACECRAFT_CLOCK_STOP_COUNT,INTEGRATION_DURATION,COMPRESSION_TYPE,...,SUB_SOLAR_LONGITUDE,SUB_SPACECRAFT_LATITUDE,SUB_SPACECRAFT_LONGITUDE,PHASE_ANGLE,EMISSION_ANGLE,SOLAR_INCIDENCE_ANGLE,CENTRAL_BODY_DISTANCE,DWELL_TIME,H_LEVEL,D_LEVEL
0,/COUVIS_0001/DATA/D1999_007/HSP1999_007_16_53.LBL,NaN,1999-01-07 16:53:07.953,1999-01-07 16:55:08.185,STAR,CO-J-UVIS-2-SSB-V1.2,1/1294419469.000,UNK,4.0,NONE,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1969-12-31 23:59:59.999999001,NaN,NaN
1,/COUVIS_0001/DATA/D1999_007/HDAC1999_007_16_31...,UHDAC,1999-01-07 16:31:07.962,1999-01-07 16:31:11.962,NaN,CO-J-UVIS-2-SSB-V1.2,1/1294418149.000,UNK,-999.0,NONE,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1970-01-01 00:00:00.000000001,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)"
2,/COUVIS_0001/DATA/D1999_007/HDAC1999_007_16_33...,UHDAC,1999-01-07 16:33:07.961,1999-01-07 16:33:11.961,NaN,CO-J-UVIS-2-SSB-V1.2,1/1294418269.000,UNK,-999.0,NONE,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1970-01-01 00:00:00.000000001,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)"
3,/COUVIS_0001/DATA/D1999_007/HDAC1999_007_16_36...,UHDAC,1999-01-07 16:36:13.960,1999-01-07 16:41:01.960,NaN,CO-J-UVIS-2-SSB-V1.2,1/1294418455.000,UNK,-999.0,NONE,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1970-01-01 00:00:00.000000072,"(0, 1, 0, 2, 0, 3, 0, 3, 0, 1, 0, 2, 0, 3, 0, 0)","(0, 0, 0, 0, 0, 0, 0, 2, 0, 3, 0, 3, 0, 3, 0, 0)"
4,/COUVIS_0001/DATA/D1999_007/HDAC1999_007_16_42...,UHDAC,1999-01-07 16:42:07.958,1999-01-07 16:42:11.958,NaN,CO-J-UVIS-2-SSB-V1.2,1/1294418809.000,UNK,-999.0,NONE,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1970-01-01 00:00:00.000000001,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)"


In [ ]:
[col for col in df.columns if "TIME" in col.upper()]

['START_TIME', 'STOP_TIME', 'DWELL_TIME']

In [ ]:
df["product_id"] = df.FILE_NAME.map(lambda x: Path(x).stem)
df["volume"] = df.FILE_NAME.map(lambda x: x.split("/")[1])

In [ ]:
df.columns

Index(['FILE_NAME', 'OBSERVATION_TYPE', 'START_TIME', 'STOP_TIME',
       'TARGET_NAME', 'DATA_SET_ID', 'SPACECRAFT_CLOCK_START_COUNT',
       'SPACECRAFT_CLOCK_STOP_COUNT', 'INTEGRATION_DURATION',
       'COMPRESSION_TYPE', 'HI_VOLTAGE_POWER_SUPPLY_STATE',
       'OCCULTATION_PORT_STATE', 'SLIT_STATE', 'TEST_PULSE_STATE', 'ODC_ID',
       'RIGHT_ASCENSION', 'DECLINATION', 'SUB_SOLAR_LATITUDE',
       'SUB_SOLAR_LONGITUDE', 'SUB_SPACECRAFT_LATITUDE',
       'SUB_SPACECRAFT_LONGITUDE', 'PHASE_ANGLE', 'EMISSION_ANGLE',
       'SOLAR_INCIDENCE_ANGLE', 'CENTRAL_BODY_DISTANCE', 'DWELL_TIME',
       'H_LEVEL', 'D_LEVEL', 'product_id', 'volume'],
      dtype='object')

In [ ]:
new_cols = ["product_id", "volume"] + list(df.columns)[:-2]

In [ ]:
df = df[new_cols]

In [ ]:
df.head()

,product_id,volume,FILE_NAME,OBSERVATION_TYPE,START_TIME,STOP_TIME,TARGET_NAME,DATA_SET_ID,SPACECRAFT_CLOCK_START_COUNT,SPACECRAFT_CLOCK_STOP_COUNT,...,SUB_SOLAR_LONGITUDE,SUB_SPACECRAFT_LATITUDE,SUB_SPACECRAFT_LONGITUDE,PHASE_ANGLE,EMISSION_ANGLE,SOLAR_INCIDENCE_ANGLE,CENTRAL_BODY_DISTANCE,DWELL_TIME,H_LEVEL,D_LEVEL
0,HSP1999_007_16_53,COUVIS_0001,/COUVIS_0001/DATA/D1999_007/HSP1999_007_16_53.LBL,NaN,1999-01-07 16:53:07.953,1999-01-07 16:55:08.185,STAR,CO-J-UVIS-2-SSB-V1.2,1/1294419469.000,UNK,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1969-12-31 23:59:59.999999001,NaN,NaN
1,HDAC1999_007_16_31,COUVIS_0001,/COUVIS_0001/DATA/D1999_007/HDAC1999_007_16_31...,UHDAC,1999-01-07 16:31:07.962,1999-01-07 16:31:11.962,NaN,CO-J-UVIS-2-SSB-V1.2,1/1294418149.000,UNK,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1970-01-01 00:00:00.000000001,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)"
2,HDAC1999_007_16_33,COUVIS_0001,/COUVIS_0001/DATA/D1999_007/HDAC1999_007_16_33...,UHDAC,1999-01-07 16:33:07.961,1999-01-07 16:33:11.961,NaN,CO-J-UVIS-2-SSB-V1.2,1/1294418269.000,UNK,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1970-01-01 00:00:00.000000001,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)"
3,HDAC1999_007_16_36,COUVIS_0001,/COUVIS_0001/DATA/D1999_007/HDAC1999_007_16_36...,UHDAC,1999-01-07 16:36:13.960,1999-01-07 16:41:01.960,NaN,CO-J-UVIS-2-SSB-V1.2,1/1294418455.000,UNK,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1970-01-01 00:00:00.000000072,"(0, 1, 0, 2, 0, 3, 0, 3, 0, 1, 0, 2, 0, 3, 0, 0)","(0, 0, 0, 0, 0, 0, 0, 2, 0, 3, 0, 3, 0, 3, 0, 0)"
4,HDAC1999_007_16_42,COUVIS_0001,/COUVIS_0001/DATA/D1999_007/HDAC1999_007_16_42...,UHDAC,1999-01-07 16:42:07.958,1999-01-07 16:42:11.958,NaN,CO-J-UVIS-2-SSB-V1.2,1/1294418809.000,UNK,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1970-01-01 00:00:00.000000001,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)"


## Duplicates

The following are duplicates in the delivered data to PDS, always in adjacent delivery volumes `COUVIS_xxxx`:

In [ ]:
df = df.sort_values(by="FILE_NAME")

In [ ]:
df[df.duplicated(subset="product_id", keep=False)].sort_values(by="product_id")

,product_id,volume,FILE_NAME,OBSERVATION_TYPE,START_TIME,STOP_TIME,TARGET_NAME,DATA_SET_ID,SPACECRAFT_CLOCK_START_COUNT,SPACECRAFT_CLOCK_STOP_COUNT,...,SUB_SOLAR_LONGITUDE,SUB_SPACECRAFT_LATITUDE,SUB_SPACECRAFT_LONGITUDE,PHASE_ANGLE,EMISSION_ANGLE,SOLAR_INCIDENCE_ANGLE,CENTRAL_BODY_DISTANCE,DWELL_TIME,H_LEVEL,D_LEVEL
32230,EUV2003_358_06_42,COUVIS_0005,/COUVIS_0005/DATA/D2003_358/EUV2003_358_06_42.LBL,UFPSCAN,2003-12-24 06:42:24.215,2003-12-24 06:46:09.215,SOLAR WIND,CO-S-UVIS-2-SPEC-V1.2,1/1450940692.128,UNK,...,-999.00000,-999.000000,-999.00000,-999.0,-999.0,-999.0,-999.00,1969-12-31 23:59:59.999999001,NaN,NaN
45080,EUV2003_358_06_42,COUVIS_0006,/COUVIS_0006/DATA/D2003_358/EUV2003_358_06_42.LBL,UFPSCAN,2003-12-24 06:42:24.215,2003-12-24 06:46:09.215,SOLAR WIND,CO-S-UVIS-2-SPEC-V1.2,1/1450940692.128,UNK,...,-999.00000,-999.000000,-999.00000,-999.0,-999.0,-999.0,-999.00,1969-12-31 23:59:59.999999001,NaN,NaN
32231,EUV2003_358_06_51,COUVIS_0005,/COUVIS_0005/DATA/D2003_358/EUV2003_358_06_51.LBL,UFPSCAN,2003-12-24 06:51:34.211,2003-12-24 06:58:14.211,SOLAR WIND,CO-S-UVIS-2-SPEC-V1.2,1/1450941242.128,UNK,...,-999.00000,-999.000000,-999.00000,-999.0,-999.0,-999.0,-999.00,1969-12-31 23:59:59.999999001,NaN,NaN
45081,EUV2003_358_06_51,COUVIS_0006,/COUVIS_0006/DATA/D2003_358/EUV2003_358_06_51.LBL,UFPSCAN,2003-12-24 06:51:34.211,2003-12-24 06:58:14.211,SOLAR WIND,CO-S-UVIS-2-SPEC-V1.2,1/1450941242.128,UNK,...,-999.00000,-999.000000,-999.00000,-999.0,-999.0,-999.0,-999.00,1969-12-31 23:59:59.999999001,NaN,NaN
32232,EUV2003_358_07_28,COUVIS_0005,/COUVIS_0005/DATA/D2003_358/EUV2003_358_07_28.LBL,UFPSCAN,2003-12-24 07:28:39.197,2003-12-24 07:31:59.197,SOLAR WIND,CO-S-UVIS-2-SPEC-V1.2,1/1450943467.128,UNK,...,-999.00000,-999.000000,-999.00000,-999.0,-999.0,-999.0,-999.00,1969-12-31 23:59:59.999999001,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60157,HDAC2004_366_22_51,COUVIS_0009,/COUVIS_0009/DATA/D2004_366/HDAC2004_366_22_51...,UHDAC,2004-12-31 22:51:04.354,2004-12-31 22:53:12.354,IAPETUS,CO-S-UVIS-2-SSB-V1.2,1/1483226223.000,UNK,...,109.74884,67.313814,353.74797,-999.0,-999.0,-999.0,126877.29,1970-01-01 00:00:00.000000032,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)"
65631,HDAC2005_274_08_11,COUVIS_0012,/COUVIS_0012/DATA/D2005_274/HDAC2005_274_08_11...,CALIB,2005-10-01 08:11:01.921,2005-10-01 08:11:05.921,SOLAR WIND,CO-S-UVIS-2-CALIB-V1.2,1/1506847172.032,UNK,...,-999.00000,-999.000000,-999.00000,-999.0,-999.0,-999.0,-999.00,1970-01-01 00:00:00.000000001,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)"
65633,HDAC2005_274_08_11,COUVIS_0013,/COUVIS_0013/DATA/D2005_274/HDAC2005_274_08_11...,CALIB,2005-10-01 08:11:01.921,2005-10-01 08:11:05.921,SOLAR WIND,CO-S-UVIS-2-CALIB-V1.2,1/1506847172.032,UNK,...,-999.00000,-999.000000,-999.00000,-999.0,-999.0,-999.0,-999.00,1970-01-01 00:00:00.000000001,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)"
65632,HDAC2005_274_08_12,COUVIS_0012,/COUVIS_0012/DATA/D2005_274/HDAC2005_274_08_12...,CALIB,2005-10-01 08:12:14.795,2005-10-01 08:14:22.795,SOLAR WIND,CO-S-UVIS-2-CALIB-V1.2,1/1506847245.000,UNK,...,-999.00000,-999.000000,-999.00000,-999.0,-999.0,-999.0,-999.00,1970-01-01 00:00:00.000000032,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)"


I ran tests to confirm that the md5sum hashs are the same for all duplicates and that the label files
only ever are different in the given data production date.

After confirming with Mark and team at RMS, we will remove the duplicates from the PDS index files, however the delivered volumes will remain unchanged.

In [ ]:
df = df.drop_duplicates(
    subset="product_id", keep="last"
)  # keep last works as I sorted by `FILE_NAME` before

In [ ]:
base_url = URL("https://opus.pds-rings.seti.org/holdings/volumes/COUVIS_0xxx/")

In [ ]:
duplicates = df[df.duplicated(subset="product_id", keep=False)]

In [ ]:
grouped = duplicates.groupby(["product_id", "FILE_NAME"]).size()

In [ ]:
grouped.head()

Series([], dtype: int64)

In [ ]:
import hashlib
import urllib


def get_remote_md5_sum(url, max_file_size=100 * 1024 * 1024):
    remote = urllib.request.urlopen(url)
    hash = hashlib.md5()

    total_read = 0
    while True:
        data = remote.read(4096)
        total_read += 4096

        if not data or total_read > max_file_size:
            break

        hash.update(data)

    return hash.hexdigest()


def read_remote_label(url):
    remote = urllib.request.urlopen(url)

    total_read = 0
    while True:
        data = remote.read(4096)
        total_read += 4096

        if not data or total_read > max_file_size:
            break

        hash.update(data)

    return hash.hexdigest()

### check for hash equality of duplicate files

In [ ]:
for pid in grouped.index.levels[0]:
    fname1, fname2 = grouped[pid].index
    url1 = base_url / str(Path(fname1[1:]).with_suffix(".DAT"))
    url2 = base_url / str(Path(fname2[1:]).with_suffix(".DAT"))
    hash1 = get_remote_md5_sum(str(url1))
    hash2 = get_remote_md5_sum(str(url2))
    if hash1 == hash2:
        print(pid, "OK")
    else:
        print(pid, "is different.")

### Check for content equality of label files

In [ ]:
for pid in grouped.index.levels[0]:
    print(pid)
    fname1, fname2 = grouped[pid].index
    url1 = base_url / fname1[1:]
    url2 = base_url / fname2[1:]
    with urllib.request.urlopen(str(url1)) as response:
        data1 = response.read().decode()
    with urllib.request.urlopen(str(url2)) as response:
        data2 = response.read().decode()
    for line1, line2 in zip(data1.splitlines(), data2.splitlines()):
        if line1 == line2:
            continue
        else:
            print(line1, "\n", line2)

EUV2003_358_06_42
PRODUCT_CREATION_TIME           = 2012-025 
 PRODUCT_CREATION_TIME           = 2012-024
EUV2003_358_06_51
PRODUCT_CREATION_TIME           = 2012-025 
 PRODUCT_CREATION_TIME           = 2012-024
EUV2003_358_07_28
PRODUCT_CREATION_TIME           = 2012-025 
 PRODUCT_CREATION_TIME           = 2012-024
EUV2003_358_07_40
PRODUCT_CREATION_TIME           = 2012-025 
 PRODUCT_CREATION_TIME           = 2012-024
EUV2003_358_07_59
PRODUCT_CREATION_TIME           = 2012-025 
 PRODUCT_CREATION_TIME           = 2012-024
EUV2003_358_09_06
PRODUCT_CREATION_TIME           = 2012-025 
 PRODUCT_CREATION_TIME           = 2012-024
EUV2003_358_10_06
PRODUCT_CREATION_TIME           = 2012-025 
 PRODUCT_CREATION_TIME           = 2012-024
EUV2003_358_10_25
PRODUCT_CREATION_TIME           = 2012-025 
 PRODUCT_CREATION_TIME           = 2012-024
EUV2003_358_11_50
PRODUCT_CREATION_TIME           = 2012-025 
 PRODUCT_CREATION_TIME           = 2012-024
EUV2003_358_12_24
PRODUCT_CREATION_TIME       

In [ ]:
df.set_index("product_id", inplace=True)

In [ ]:
filter = df.index.map(lambda x: x[:3] in ["EUV", "FUV"])

In [ ]:
pds_euv_fuv = df[filter]

In [ ]:
pds_euv_fuv.head()

,volume,FILE_NAME,OBSERVATION_TYPE,START_TIME,STOP_TIME,TARGET_NAME,DATA_SET_ID,SPACECRAFT_CLOCK_START_COUNT,SPACECRAFT_CLOCK_STOP_COUNT,INTEGRATION_DURATION,...,SUB_SOLAR_LONGITUDE,SUB_SPACECRAFT_LATITUDE,SUB_SPACECRAFT_LONGITUDE,PHASE_ANGLE,EMISSION_ANGLE,SOLAR_INCIDENCE_ANGLE,CENTRAL_BODY_DISTANCE,DWELL_TIME,H_LEVEL,D_LEVEL
product_id,,,,,,,,,,,,,,,,,,,,,
FUV1999_007_16_57,COUVIS_0001,/COUVIS_0001/DATA/D1999_007/FUV1999_007_16_57.LBL,USTARE,1999-01-07 16:57:21.952,1999-01-07 17:00:57.952,NaN,CO-J-UVIS-2-SPEC-V1.2,1/1294419723.000,UNK,4.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1969-12-31 23:59:59.999999001,NaN,NaN
FUV1999_007_17_01,COUVIS_0001,/COUVIS_0001/DATA/D1999_007/FUV1999_007_17_01.LBL,USTARE,1999-01-07 17:01:11.950,1999-01-07 17:04:47.950,NaN,CO-J-UVIS-2-SPEC-V1.2,1/1294419953.000,UNK,4.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1969-12-31 23:59:59.999999001,NaN,NaN
EUV1999_007_17_05,COUVIS_0001,/COUVIS_0001/DATA/D1999_007/EUV1999_007_17_05.LBL,USTARE,1999-01-07 17:05:01.949,1999-01-07 17:08:37.949,NaN,CO-J-UVIS-2-SPEC-V1.2,1/1294420183.000,UNK,4.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1969-12-31 23:59:59.999999001,NaN,NaN
EUV1999_007_17_08,COUVIS_0001,/COUVIS_0001/DATA/D1999_007/EUV1999_007_17_08.LBL,USTARE,1999-01-07 17:08:51.947,1999-01-07 17:12:27.947,NaN,CO-J-UVIS-2-SPEC-V1.2,1/1294420413.000,UNK,4.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1969-12-31 23:59:59.999999001,NaN,NaN
FUV1999_010_07_16,COUVIS_0001,/COUVIS_0001/DATA/D1999_010/FUV1999_010_07_16.LBL,USTARE,1999-01-10 07:16:50.522,1999-01-10 09:04:10.522,NaN,CO-J-UVIS-2-SPEC-V1.2,1/1294644093.000,UNK,10.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1969-12-31 23:59:59.999999001,NaN,NaN


In [ ]:
pds_euv_fuv.query("index=='EUV2002_198_04_31'")

,volume,FILE_NAME,OBSERVATION_TYPE,START_TIME,STOP_TIME,TARGET_NAME,DATA_SET_ID,SPACECRAFT_CLOCK_START_COUNT,SPACECRAFT_CLOCK_STOP_COUNT,INTEGRATION_DURATION,...,SUB_SOLAR_LONGITUDE,SUB_SPACECRAFT_LATITUDE,SUB_SPACECRAFT_LONGITUDE,PHASE_ANGLE,EMISSION_ANGLE,SOLAR_INCIDENCE_ANGLE,CENTRAL_BODY_DISTANCE,DWELL_TIME,H_LEVEL,D_LEVEL
product_id,,,,,,,,,,,,,,,,,,,,,
EUV2002_198_04_31,COUVIS_0004,/COUVIS_0004/DATA/D2002_198/EUV2002_198_04_31.LBL,USTARE,2002-07-17 04:31:59.571,2002-07-17 05:27:19.571,STAR,CO-S-UVIS-2-CUBE-V1.2,1/1405572559.096,UNK,332.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1969-12-31 23:59:59.999999001,NaN,NaN


In [ ]:
pds_euv_fuv[pds_euv_fuv.index.str.contains("EUV2002_198")]

,volume,FILE_NAME,OBSERVATION_TYPE,START_TIME,STOP_TIME,TARGET_NAME,DATA_SET_ID,SPACECRAFT_CLOCK_START_COUNT,SPACECRAFT_CLOCK_STOP_COUNT,INTEGRATION_DURATION,...,SUB_SOLAR_LONGITUDE,SUB_SPACECRAFT_LATITUDE,SUB_SPACECRAFT_LONGITUDE,PHASE_ANGLE,EMISSION_ANGLE,SOLAR_INCIDENCE_ANGLE,CENTRAL_BODY_DISTANCE,DWELL_TIME,H_LEVEL,D_LEVEL
product_id,,,,,,,,,,,,,,,,,,,,,
EUV2002_198_00_47,COUVIS_0004,/COUVIS_0004/DATA/D2002_198/EUV2002_198_00_47.LBL,UFPSCAN,2002-07-17 00:47:28.041,2002-07-17 02:00:58.041,SOLAR WIND,CO-S-UVIS-2-SPEC-V1.2,1/1405559087.192,UNK,49.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1969-12-31 23:59:59.999999001,NaN,NaN
EUV2002_198_02_59,COUVIS_0004,/COUVIS_0004/DATA/D2002_198/EUV2002_198_02_59.LBL,UFPSCAN,2002-07-17 02:59:45.860,2002-07-17 03:26:42.860,SOLAR WIND,CO-S-UVIS-2-SPEC-V1.2,1/1405567025.160,UNK,49.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1969-12-31 23:59:59.999999001,NaN,NaN
EUV2002_198_03_26,COUVIS_0004,/COUVIS_0004/DATA/D2002_198/EUV2002_198_03_26.LBL,USTARE,2002-07-17 03:26:28.349,2002-07-17 04:20:20.349,STAR,CO-S-UVIS-2-CUBE-V1.2,1/1405568628.032,UNK,1.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1969-12-31 23:59:59.999999001,NaN,NaN
EUV2002_198_04_31,COUVIS_0004,/COUVIS_0004/DATA/D2002_198/EUV2002_198_04_31.LBL,USTARE,2002-07-17 04:31:59.571,2002-07-17 05:27:19.571,STAR,CO-S-UVIS-2-CUBE-V1.2,1/1405572559.096,UNK,332.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1969-12-31 23:59:59.999999001,NaN,NaN
EUV2002_198_05_28,COUVIS_0004,/COUVIS_0004/DATA/D2002_198/EUV2002_198_05_28.LBL,USTARE,2002-07-17 05:28:11.547,2002-07-17 06:23:31.547,STAR,CO-S-UVIS-2-CUBE-V1.2,1/1405575931.096,UNK,332.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1969-12-31 23:59:59.999999001,NaN,NaN
EUV2002_198_06_24,COUVIS_0004,/COUVIS_0004/DATA/D2002_198/EUV2002_198_06_24.LBL,USTARE,2002-07-17 06:24:23.399,2002-07-17 07:19:43.399,STAR,CO-S-UVIS-2-CUBE-V1.2,1/1405579303.064,UNK,332.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1969-12-31 23:59:59.999999001,NaN,NaN
EUV2002_198_07_20,COUVIS_0004,/COUVIS_0004/DATA/D2002_198/EUV2002_198_07_20.LBL,USTARE,2002-07-17 07:20:35.375,2002-07-17 08:15:55.375,STAR,CO-S-UVIS-2-CUBE-V1.2,1/1405582675.064,UNK,332.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1969-12-31 23:59:59.999999001,NaN,NaN
EUV2002_198_08_16,COUVIS_0004,/COUVIS_0004/DATA/D2002_198/EUV2002_198_08_16.LBL,USTARE,2002-07-17 08:16:47.351,2002-07-17 09:12:07.351,STAR,CO-S-UVIS-2-CUBE-V1.2,1/1405586047.064,UNK,332.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1969-12-31 23:59:59.999999001,NaN,NaN
EUV2002_198_09_12,COUVIS_0004,/COUVIS_0004/DATA/D2002_198/EUV2002_198_09_12.LBL,USTARE,2002-07-17 09:12:59.328,2002-07-17 10:08:19.328,STAR,CO-S-UVIS-2-CUBE-V1.2,1/1405589419.064,UNK,332.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1969-12-31 23:59:59.999999001,NaN,NaN


In [ ]:
spica.groupby("DATE").size()

DATE
1999-01-16    52
1999-01-17    50
2001-04-03     6
2001-10-26    14
2001-10-27     4
              ..
2016-02-13     8
2016-07-13    36
2017-01-13     2
2017-06-21    14
2017-06-22    12
Length: 75, dtype: int64

In [ ]:
spica.query("DATE=='2001-04-03'")

,filename,slit,nx,ny,nz,int,odcid,x1,x2,y1,y2,name,DET,DATETIME,DATE
94,EUV2001_093_08_35_28,2,1024,64,29,60,93,0,1023,0,63,alp vir,EUV,2001-04-03 08:35:28,2001-04-03
95,EUV2001_093_09_05_28,2,1024,64,29,60,94,0,1023,0,63,alp vir,EUV,2001-04-03 09:05:28,2001-04-03
96,EUV2001_093_16_52_33,2,1024,64,99,36,95,0,1023,0,63,alp vir,EUV,2001-04-03 16:52:33,2001-04-03
757,FUV2001_093_08_35_28,2,1024,64,29,60,93,0,1023,0,63,alp vir,FUV,2001-04-03 08:35:28,2001-04-03
758,FUV2001_093_09_05_28,1,1024,64,29,60,94,0,1023,0,63,alp vir,FUV,2001-04-03 09:05:28,2001-04-03
759,FUV2001_093_16_52_33,2,1024,64,99,36,95,0,1023,0,63,alp vir,FUV,2001-04-03 16:52:33,2001-04-03


In [ ]:
fname = "EUV2001_093_08_35_28"

In [ ]:
s = pds_euv_fuv.index

In [ ]:
s[s.str.startswith("EUV2001_093")]

Index(['EUV2001_093_00_07', 'EUV2001_093_01_03', 'EUV2001_093_02_55',
       'EUV2001_093_03_07', 'EUV2001_093_04_03', 'EUV2001_093_08_35',
       'EUV2001_093_09_05', 'EUV2001_093_16_52', 'EUV2001_093_17_52',
       'EUV2001_093_18_54'],
      dtype='object', name='FILE_NAME')

In [ ]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,9,1999-01-07 10:05:02.093,1999-01-07 10:08:34.0,EUV,NaN,EUV1999-01-07 17:05:02.000,NaN,USTARE,to evaluate EUV and FUV functions.,7,32,0,0,0.0,0.0,1.0,0.0
1,10,1999-01-07 10:08:52.09,1999-01-07 10:12:24.0,EUV,NaN,EUV1999-01-07 17:08:52.000,NaN,USTARE,to evaluate EUV and FUV functions.,8,32,0,1,0.0,0.0,0.0,0.0
2,19,1999-01-10 00:16:50.06,1999-01-10 02:04:00.0,EUV,NaN,EUV1999-01-10 07:16:50.000,NaN,USTARE,to evaluate crosstalk between S/C subsystems a...,9,80,1,1,0.0,0.0,0.0,0.0
3,20,1999-01-10 02:09:45.013,1999-01-10 02:46:15.0,EUV,NaN,EUV1999-01-10 09:09:45.000,NaN,USTARE,to evaluate crosstalk between S/C subsystems a...,9,80,1,1,0.0,0.0,0.0,0.0
4,172,1999-01-16 12:47:15.356,1999-01-16 12:48:15.0,EUV,NaN,EUV1999-01-16 19:47:15.000,NaN,USTARE,"to evaluate EUV, FUV response to the star Alph...",12,480,1,1,0.0,2.0,0.0,0.0
